1.a
$B(Pokemon) = |Pokemon|/|block| = 10^5/10 = 10^4$ blocks to store Pokemon

1.b
$B(Trainer) = |Trainer|/|block| = 5*10^3/10 = 5*10^2$ blocks to store Trainer


#### 21 memory blocks available to sort Pokemon by tid using external merge sort algorithm ####

1.c
The number of Level-0 runs required will be $\lceil \frac{B(Pokemon)}{M} \rceil = \lceil \frac{10^4}{21} \rceil = 477$ runs

1.d
The number of level-1 runs required will be $\lceil \frac{Num of level-0 runs}{M-1} \rceil = \lceil \frac{477}{20} \rceil = 24$ runs

1.e
The number of level-2 runs required will be $\lceil \frac{Num of level-1 runs}{M-1} \rceil = \lceil \frac{24}{20} \rceil = 2$ runs

1.f
The number of level-3 runs required will be $\lceil \frac{Num of level-2 runs}{M-1} \rceil = \lceil \frac{2}{20} \rceil = 1$ run

1.g
The number of passes required will be $\lceil \log_{M-1}\frac{B(R)}{M} \rceil + 1  = \lceil \log_{20}\frac{10^4}{21} \rceil + 1 = 4$ passes

#### Optimized multi-pass sort merge algorithm covered in lecture to compute Pokemon full join Trainer. Trainer is already sorted by join attribute (tid) so we only need to merge it together with the Pokemon runs during the last sorting pass of Pokemon (as long as there is sufficient memory) ####

1.h
The first 3 passes will each take $2*B(Pokemon)$ I/Os to read and write the Pokemon runs (since Trainer is sorted). The last pass will take $B(Pokemon) + B(Trainer)$ I/Os to read the Pokemon runs and Trainer runs. The total number of I/Os will be $3*2*B(Pokemon) + B(Pokemon) + B(Trainer) = 7*10^4 + 5*10^2 = 70500$ I/Os

#### Suppose instead we hae a total of 26 memory blocks available to perform the two-pass hash join algorithm to compute Pokemon full join Trainer. (Any block used for buffering output must come from these blocks too.) For simplicity, assume that all trainers have an equal number of Pokemon, there are no wild Pokemon, and w have a perfect has function. ####

1.i
The number of partitions per table will be $M-1 = 25$ partitions per table

1.j
The size of a Pokemon partition will be $\lceil \frac{B(Pokemon)}{M-1} \rceil = \lceil \frac{10^4}{26-1} \rceil = 400$

1.k
The size of a Trainer partition will be $\lceil \frac{B(Trainer)}{M-1} \rceil = \lceil \frac{5*10^2}{26-1} \rceil = 20$

1.l
We want to use the smaller partition, so we will use the Trainer partitions.

1.m
The minimum value of M required for the two-pass hash join algorithm to work is 
$M - 1 > \sqrt{min(B(Pokemon), B(Trainer))}$ $ \\ $
So $M > \sqrt{500} + 1 $ $ \\ $
So $M > 23$ $ \\ $
$ M = 24 $

At level 0 we have 477 sets of sorted Pokemon "partitions". So in pass 0, we continue to do what the optimized SMJ does. Instead of merging each one of these sets together in pass 1, 2 and 3, we can simply feed in a partition and a trainer together and output the result.

We would have $2B(Pokemon) + (B(Pokemon)+477*B(Trainer)) = 2*10^4 + (10^4+477*5*10^2) = 268500$ I/Os

This seems like a lot, but we can further optimize this by loading in 19 partitions (since we need to reserve 1 for the trainer and 1 for the buffer).

This would reduce the number of I/Os to $2B(Pokemon) + (B(Pokemon)+\lceil 477/19 \rceil * B(Trainer)) = 2*10^4 + (10^4 + 26*5*10^2) =  43000 $ I/Os

Recap: the first pass costs $2*10^4$ while we merge Pokemon together, and the second pass costs $10^4$ when we sort Pokemon and $\lceil 477/19 \rceil * 5 * 10^2$ when we merge Pokemon and Trainer together. The total number of I/Os is $2*10^4 + 10^4 + \lceil 477/19 \rceil * 5 * 10^2 = 43000$ I/Os

Question 2.

$|Pokemon| = 10^5 \\ 
|\pi_{species}(Pokemon)| = 10^3 \\
|\pi_{tid}(Pokemon)| = 5*10^3 \\
|Gym| = 500 \\
|\pi_{name}Gym| = 500 \\
|Placement| = 4000 \\
|\pi_{gid}Placement| = 500 \\ $

2.a 
$|\sigma_{species = 'Zubat'}Pokemon| \approx \frac{|Pokemon|}{|\pi_{species}{Pokemon}|} = 10^5/10^3 = 100$

2.b $ |\sigma_{name = 'Stone Arch'}Gym| = \frac{|Gym|}{|\pi_{name}{Gym}|} = 500/500 = 1 \\
|(\sigma_{name = 'Stone Arch'}Gym) \bowtie Placement| = \frac{|\sigma_{name = 'Stone Arch'}Gym||Placcement|}{max(|\pi_{gid} (\sigma_{name = 'Stone Arch'}Gym)|, |\pi_{gid} Placement|)} = \frac{1*4000}{500} = 8$

2.c
$|Pokemon \bowtie Placement| = \frac{|Pokemon||Placement|}{max(|\pi_{pid} Pokemon|, |\pi_{pid} Placement|)} = \frac{10^5 * 4000}{10^5} = 4000$

2.d
$|\sigma_{species = 'Zubat'} (Pokemon \bowtie Placement)| = \frac{|Pokemon \bowtie Placement|}{|\pi_{species} (Pokemon \bowtie Placement)|} = \frac{|Pokemon \bowtie Placement|}{|\pi_{species} (Pokemon)|} = \frac{4000}{10^3} = 4$

2.e
$|(\sigma_{species = 'Zubat'} Pokemon) \bowtie Placement| = \frac{|(\sigma_{species = 'Zubat'} Pokemon)||Placement|}{max(|\pi_{pid} (\sigma_{species = 'Zubat'} Pokemon)|, |\pi_{pid} Placement|)} = \frac{100*4000}{|\pi_{pid} Placement|} = \frac{100*4000}{4000} = 100$

2.f
In 2d, we make the assumption for preservation of value sets for Pokemon.species, that is: $\pi_{species} (Pokemon \bowtie Placement) = \pi_{species} (Pokemon) \\ $
This is quite realistic, as we know that Pokemon can only be in one placement at a time. $ \\ $
In 2e, we make the assumption for containment of value sets for pid, that is: every tuple in Placement is joined to some tuple in $(\sigma_{species = 'Zubat'} Pokemon) \\ $
This is not realistic, as $(\sigma_{species = 'Zubat'} Pokemon)$ will not contain every Pokemon that is in Placement (since there are only Zubats).

Therefore, 2d is more realistic.


Question 3

Each block can hold 10 Pokemon rows or 20 Placement rows.
All rows in each table are stored compactly on disk.
We have only 4 memory blocks for query processing. Final result must be written to disk, so a block needs to be used to buffer output.

3.a
Consider a very inefficient query plan, where we compute  using an optimized block-based nested-loop join algorithm. It gets two blocks of Pokemon rows in memory at a time, and uses a third memory block to scan Placement to look for join result rows to output, which are buffered using the fourth memory block. The condition species = 'Zubat' is checked right before we put a result row into the output buffer block. What is the total number of block I/O’s this query plan uses (not counting the cost of writing the final query result to disk)?

20 pokemon rows are gotten at a time. It will take 10^5/10 = 5000 iterations to get all the pokemon rows. For each iteration we are reading two blocks so this will take 2 I/Os per iteration. So the total number of I/Os will be 5000*2 = 10000 I/Os

For each iteration of getting 5000, we will need to scan through the entire Placement table to find the rows that match the pokemon rows. This will take 4000/20 = 200 iterations where we read the Placement table so 200 I/Os per iteration. In total, we will need 200*5000 = 1000000 I/Os

When we stream together the Placement and Pokemon rows, we also check if it matches species = 'Zubat'. This costs 0 I/Os.

In total, we need $10000 + 1000000 = 1010000$ I/Os

3.b
When we read the Pokemon blocks in, we should check to see if species = 'Zubat'. This will continue to cost 10000 I/Os as we need to stream all of Pokemon in. As we find each Zubat, we will load it into our two Pokemon memory blocks. In the worst case we have 100 Zubats, which is 10 blocks of Zubats. Each time we encounter 2 blocks of Zubat tuples, we will need to check those tuples match up with Placement. This will cost us 200 I/Os per iteration. In total, we will need 200*5 = 1000 I/Os.

In total, we need $10000 + 1000 = 11000$ I/Os

3.c 
It costs us 5 I/Os to look up the index for the Zubat species. Since there are 100 Zubats at most, they can span 4 blocks (of 30 each). This means it costs us an additional 4 I/Os to load the Zubats into memory. In total, this costs us 9 I/Os.

To load in each Zubat into memory will cost us 1 I/O per Zubat. In total, this costs us 100 I/Os. We will load 1 memory block of Zubat at a tie into memory.

In another memory block, we will store the root of the B+tree index for Placement.
This costs 1 I/O.

It costs us 2 I/Os to lookup a placement from the primary B+tree index.
Since there are 100 Zubats, this will cost us 100*2 = 200 I/Os.

In total we need $1 + 200 + 100 + 5 + 4 = 310$ I/Os